# Trabalho Prático 2 - ACO para o problema do JSS
## Júlia Fonseca de Sena - 2018054508

In [1]:
import numpy as np
import random
from collections import defaultdict
import matplotlib.pyplot as plt

## Leitura do arquivo de entrada

In [2]:
entrada = 'la40.txt'
arq = open(entrada, 'r')
arq_data = np.array([])
for x in arq: #Guarda todas as linhas do arquivo em um array
    arq_data = np.append(arq_data, x.strip())

print('Linhas do arquivo:')    
print(arq_data)

Linhas do arquivo:
['Lawrence 15x15 instance (Table 10, instance 5); also called (seti5) or (I5)'
 '15 15'
 '9 65 10 28  4 74 12 33  2 51 14 75  5 73  8 32  6 13  3 81  1 35  7 59 13 38 11 55  0 27'
 '0 64  1 53 11 83  2 33  4  6  9 52 14 72  8  7 13 90 12 21  6 23  3 10 10 39  5 49  7 72'
 '14 73  3 82  1 23 12 62  6 88  5 21  8 65 11 70  7 53 10 81  2 93 13 77  0 61  9 28  4 78'
 '1 12  6 51  7 33  4 15 14 72 10 98  9 94  5 12 11 42  2 24 13 15  8 28  3  6 12 99  0 41'
 '12 97  5  7  9 96  4 15 14 73 13 43  0 32  8 22 11 42  1 94  2 23  7 86  6 78 10 24  3 31'
 '1 72  5 88  2 93 13 13  4 44 14 66  6 63  7 14  9 67 10 17 11 85  0 35  3 68 12  5  8 49'
 '9 15  7 82  6 21 14 53  3 72 13 49  2 99  4 26 12 56  8 45  1 68 10 51  0  8  5 27 11 96'
 '3 54  7 24  4 14  8 38  5 36  2 52 14 55 12 37 11 48  0 93 13 60 10 70  1 23  6 23  9 83'
 '3 12  8 69  6 26  9 23 14 28  1 82  5 33  4 45 13 64  7 15 11  9 12 73 10 59  2 37  0 62'
 '0 87  5 12  7 80  4 50 10 48 12 90  1 72 13 24  6 14  8 71 11

In [3]:
if not arq_data[0][0].isdigit(): #No caso da primeira linha da instância ser o título (não foi especificado)
    arq_data = np.delete(arq_data, 0)

#Lê a primeira linha
data = arq_data[0].split() #Lista de palavras da string
num_jobs = int(data[0]) #Primeiro o número de jobs
num_maq = int(data[1]) #Depois o número de máquinas
arq_data = np.delete(arq_data, 0) #Já processada, retira linha do array

print('Número de máquinas e jobs:', num_maq, num_jobs)

Número de máquinas e jobs: 15 15


In [4]:
jobs = np.ndarray(shape=(num_jobs,num_maq), dtype=tuple) #Matriz com dados dos trabalhos

for i in range(num_jobs): #Para todos os jobs
    data = arq_data[i].split() #Lista de palavras da linha
    index_data = 0 #Índice na lista de palavras
    for j in range(num_maq): #Para toda operação
        jobs[i][j] = (int(data[index_data]), int(data[index_data + 1])) #Pega dados sequencialmente
        index_data += 2 #Atualiza índice de palavras

print('Matriz dos jobs:')
jobs

Matriz dos jobs:


array([[(9, 65), (10, 28), (4, 74), (12, 33), (2, 51), (14, 75), (5, 73),
        (8, 32), (6, 13), (3, 81), (1, 35), (7, 59), (13, 38), (11, 55),
        (0, 27)],
       [(0, 64), (1, 53), (11, 83), (2, 33), (4, 6), (9, 52), (14, 72),
        (8, 7), (13, 90), (12, 21), (6, 23), (3, 10), (10, 39), (5, 49),
        (7, 72)],
       [(14, 73), (3, 82), (1, 23), (12, 62), (6, 88), (5, 21), (8, 65),
        (11, 70), (7, 53), (10, 81), (2, 93), (13, 77), (0, 61), (9, 28),
        (4, 78)],
       [(1, 12), (6, 51), (7, 33), (4, 15), (14, 72), (10, 98), (9, 94),
        (5, 12), (11, 42), (2, 24), (13, 15), (8, 28), (3, 6), (12, 99),
        (0, 41)],
       [(12, 97), (5, 7), (9, 96), (4, 15), (14, 73), (13, 43), (0, 32),
        (8, 22), (11, 42), (1, 94), (2, 23), (7, 86), (6, 78), (10, 24),
        (3, 31)],
       [(1, 72), (5, 88), (2, 93), (13, 13), (4, 44), (14, 66), (6, 63),
        (7, 14), (9, 67), (10, 17), (11, 85), (0, 35), (3, 68), (12, 5),
        (8, 49)],
       [(9, 15)

## Parâmetros

In [5]:
#Inicialização dos parâmetros
num_ciclos = 70
num_ants = 2*num_jobs + (num_maq-1)*num_jobs + num_jobs*num_maq
feromonio_inicial = 0.02

alfa = 0.4
beta = 0.6
evaporacao = 0.7
Q = 100

## Contadores

In [6]:
otimo = 1222
encontrou_otimo = 0

## Grafo Disjuntivo

In [7]:
class No: #Cada nó é uma operação
    def __init__(self, job, maquina, tempo, boolean = False):
        self.job = job #Número do job
        self.maq = maquina #Número da máquina
        self.tempo = tempo #Tempo de processamento
        
        self.direcionadas = np.array([]) #Arestas direcionadas (só é mais que uma para o nó inicial)
        self.feromonio_dir = np.array([]) #Feromônio das direcionadas
        self.nao_direcionadas = np.array([]) #Arestas não direcionadas
        self.feromonio_nao_dir = np.array([]) #Feromônio das não direcionadas

        self.pode_usar = boolean #Verifica se operação já pode ser feita (anteriores concluídas)
        
    def __repr__(self): #Impressão
        return f'({self.job}, {self.maq}, {self.tempo})'
        
    def adiciona_direcionada(self, no, feromonio = feromonio_inicial): #Adiciona uma aresta direcionada
        self.direcionadas = np.append(self.direcionadas, no)
        self.feromonio_dir = np.append(self.feromonio_dir, feromonio)
    
    def adiciona_nao_direcionada(self, no, feromonio = feromonio_inicial): #Adiciona uma aresta não direcionada
        if no not in self.nao_direcionadas:
            self.nao_direcionadas = np.append(self.nao_direcionadas, no)
            self.feromonio_nao_dir = np.append(self.feromonio_nao_dir, feromonio)
            
        if self not in no.nao_direcionadas: #Em ambos nós
            no.nao_direcionadas = np.append(no.nao_direcionadas, self)
            no.feromonio_nao_dir = np.append(no.feromonio_nao_dir, feromonio)
        
class Grafo: #Grafo com relação das operações
    def __init__(self):
        self.inicio = No(-1, -1, -1) #Valores arbitrários para nós de início e fim
        self.fim = No(-1, -1, -1)
    
    def completa_direcionadas(self): #Coloca arestas direcionadas para operações de um mesmo job
        for job in range(0, num_jobs):
            atual = No(job, jobs[job][0][0], jobs[job][0][1], boolean = True)
            self.inicio.adiciona_direcionada(atual) #Do início para a primeira operação
            
            for op in range(1, num_maq): #Coloca as operações na sequência (cada nó aponta para a operação seguinte)
                prox = No(job, jobs[job][op][0], jobs[job][op][1])
                atual.adiciona_direcionada(prox)
                atual = prox
            
            atual.adiciona_direcionada(self.fim) #Última aponta para o fim
    
    def completa_nao_direcionadas(self): #Coloca arestas não direcionadas para operações de uma mesma máquina
        for job in range(num_jobs): #Começando pelo primeiro job
            no_atual = self.inicio.direcionadas[job]
            for op in range(num_maq): #Para cada operação
                maq = no_atual.maq
                for other_job in range(job+1, num_jobs): #Passa pelos jobs seguintes
                    no_comp = self.inicio.direcionadas[other_job]
                    for other_op in range(num_maq): #Para todas as operações
                        maq_comp = no_comp.maq
                        if maq_comp == maq: #Se a máquina for igual
                            no_atual.adiciona_nao_direcionada(no_comp) #Cria aresta não direcionada
                            break
                        no_comp = no_comp.direcionadas[0] #Próxima do job para comparar
                no_atual = no_atual.direcionadas[0] #Próxima do job para achar não direcionadas
                            
    def constroi_grafo(self): #Chama as funções de construção do grafo
        self.completa_direcionadas()
        self.completa_nao_direcionadas()

    def reseta_pode_usar(self): #Reseta pode_usar para valores iniciais (chamado depois da construção de cada formiga)
        for job in self.inicio.direcionadas: #A partir da segunda operação de cada job
            op = job
            while op != self.fim: #Enquanto não chegou no fim
                op = op.direcionadas[0]
                op.pode_usar = False #Não é alcançável

In [8]:
G = Grafo()
G.constroi_grafo() #Constrói grafo global

## Formigas

#### Cálculo da probabilidade das arestas

In [9]:
def lrt(arestas): #Heurística LRT
    tempo_restante = np.array([]) #Salva o tempo restante dos jobs de cada aresta
    for a in arestas:
        no = a[0] #Nó é o primeiro elemento da tupla
        soma = 0
        while no != G.fim: #Enquanto não chegou no fim
            soma += no.tempo #Soma tempo ao total
            no = no.direcionadas[0] #Próxima operação do job
        tempo_restante = np.append(tempo_restante, soma) #Coloca na lista o tempo restante
    
    N = np.argsort(tempo_restante) #Retorna nos mesmos índices quais tempos são maiores
    N += 1 #Adiciona 1 para não ter 0
    
    return N

def probabilidade(arestas): #Retorna a probabilidade de escolher cada aresta
    N = lrt(arestas) #Peso de cada aresta pelo índice LPT
    
    lista_prob = [] #Lista de probabilidade de cada aresta
    soma = 0
    for i in range(len(arestas)):
        lista_prob.append((arestas[i][1] ** alfa) * (N[i] ** beta)) #De cada aresta (feromônio ** 0.2) * (heuristica ** 0.8)
        soma += (arestas[i][1] ** alfa) * (N[i] ** beta) #Soma de todas as arestas

    return (lista_prob / soma) #Divide todas pela soma

def escolha(arestas): #Escolhe uma aresta pra seguir
    probs = probabilidade(arestas) #Lista de probabilidades
    
    p_escolha = random.random() #Número aleatório entre 0 e 1
    if p_escolha == 1: #Se for igual a 1
        index_escolha = len(possiveis) - 1 #Já escolhe o último
    else:
        index_escolha = 0 
        while p_escolha > probs[index_escolha]: #Se for menor que a probabilidade, não está em seu intervalo
            p_escolha -= probs[index_escolha] #Diminui a probabilidade atual do número aleatório escolhido
            index_escolha += 1 #Passa para próxima aresta
    
    return index_escolha

#### Classe das formigas

In [10]:
class Ant: #Formiga
    def __init__(self):
        self.tabu = np.array([]) #Sequência de nós
        self.makespan = 0 #Makespan
        self.contribuicao = 0 #Contribuição no feromônio
    
    def __repr__(self): #Impressão
        return f'{self.tabu}, {self.makespan}, {self.contribuicao}\n'
    
    def constroi_solucao(self): #Constroi solução
        index = escolha(list(zip(G.inicio.direcionadas, G.inicio.feromonio_dir))) #Escolha inicial
        job_possiveis = np.copy(G.inicio.direcionadas) #Operações possíveis de cada job
        no_atual = G.inicio.direcionadas[index] #Primeiro nó escolhido
        
        while len(job_possiveis) > 0: #Enquanto ainda houver operações possíveis para algum job
            if no_atual.direcionadas[0] != G.fim: #Se direcionada não apontar para o fim
                job_possiveis[np.where(job_possiveis == no_atual)[0]] = no_atual.direcionadas[0] #Atualiza operação do job para a próxima
                no_atual.direcionadas[0].pode_usar = True #Pode usar a próxima
            else: #Se é o fim do grafo
                job_possiveis = np.delete(job_possiveis, np.where(job_possiveis == no_atual)) #Tira job do vetor
                    
            self.tabu = np.append(self.tabu, no_atual) #Coloca o nó atual no tabu
            
            possiveis = [] #Lista com arestas possíveis para seguir
            if no_atual.direcionadas[0].pode_usar == True: #Se pode seguir a direcionada (não pode se for fim do grafo)
                possiveis.append((no_atual.direcionadas[0], no_atual.feromonio_dir[0])) #Coloca na lista de possíveis
                    
            for i in range(num_jobs - 1): #Coloca as não direcionadas que já podem ser processadas
                no = no_atual.nao_direcionadas[i]
                if no.pode_usar == True and no not in self.tabu:
                    possiveis.append((no, no_atual.feromonio_nao_dir[i]))
                
            if len(possiveis) == 0 and len(self.tabu) < num_jobs*num_maq: #Se não há alcançáveis do nó mas não passou por todos ainda
                index_escolha = random.randint(0, len(job_possiveis)-1) #Escolhe um aleatório entre as operações possíveis
                no_atual = job_possiveis[index_escolha]
            elif len(possiveis) > 0: #Se há arestas possíveis
                index_escolha = escolha(possiveis) #Escolhe de acordo com as probabilidades da aresta
                no_atual = possiveis[index_escolha][0]
            
        G.reseta_pode_usar() #Depois de construir, reseta as operações alcançáveis
    
    def calcula_makespan(self): #Calcula o makespan da solução encontrada
        jobs_tempo = []
        for i in range(num_jobs): #O tempo final atual de cada job
            jobs_tempo.append(0) #Inicial é 0
        
        maquinas_tempo = defaultdict(list) #O tempo final de cada máquina
        
        for op in self.tabu: #Para a sequência de operações no tabu
            comeco_computacao = max(maquinas_tempo[op.maq], default = 0) 
            maquinas_tempo[op.maq].append(op.tempo)
            fim_computacao = comeco_computacao + op.tempo
            jobs_tempo[op.job] += fim_computacao
        
        self.makespan = max(jobs_tempo) #O job que com maior tempo de fim é 
        
    def calcula_adicao_feromonio(self): #Calcula a contribuição para as arestas da solução
        self.contribuicao = Q/self.makespan
    
    def constroi(self): #Chama as funções para a construção da formiga
        self.constroi_solucao()
        self.calcula_makespan()
        self.calcula_adicao_feromonio()

formiga = Ant()
formiga.constroi()
print(formiga)

[(12, 0, 49) (9, 0, 87) (9, 5, 12) (9, 7, 80) (9, 4, 50) (9, 10, 48)
 (11, 10, 92) (13, 10, 98) (13, 12, 34) (13, 13, 52) (13, 4, 26)
 (13, 1, 28) (3, 1, 12) (3, 6, 51) (3, 7, 33) (3, 4, 15) (3, 14, 72)
 (2, 14, 73) (2, 3, 82) (2, 1, 23) (2, 12, 62) (9, 12, 90) (9, 1, 72)
 (5, 1, 72) (14, 1, 70) (14, 10, 17) (14, 6, 90) (2, 6, 88) (2, 5, 21)
 (5, 5, 88) (5, 2, 93) (10, 2, 54) (10, 0, 22) (1, 0, 64) (1, 1, 53)
 (1, 11, 83) (1, 2, 33) (1, 4, 6) (1, 9, 52) (1, 14, 72) (11, 14, 36)
 (11, 4, 22) (11, 9, 9) (6, 9, 15) (6, 7, 82) (6, 6, 21) (10, 6, 61)
 (10, 4, 46) (10, 3, 73) (7, 3, 54) (8, 3, 12) (13, 3, 39) (11, 3, 47)
 (11, 1, 77) (11, 12, 79) (11, 13, 36) (12, 13, 83) (12, 3, 73)
 (12, 6, 82) (12, 1, 82) (12, 14, 92) (6, 14, 53) (6, 3, 72) (6, 13, 49)
 (9, 13, 24) (9, 6, 14) (9, 8, 71) (2, 8, 65) (8, 8, 69) (13, 8, 80)
 (13, 5, 29) (10, 5, 16) (10, 12, 6) (10, 9, 94) (10, 14, 93) (10, 13, 67)
 (5, 13, 13) (5, 4, 44) (5, 14, 66) (5, 6, 63) (5, 7, 14) (7, 7, 24)
 (7, 4, 14) (7, 8, 38) (7, 

## Adição de feromônio

In [11]:
def evaporacao_todas_arestas(): #Evapora para todas as arestas
    G.inicio.feromonio_dir *= evaporacao #Multiplica pela evaporação as iniciais
    G.inicio.feromonio_nao_dir *= evaporacao 
    
    for job in G.inicio.direcionadas: #Para todos os jobs
        no_atual = job
        while no_atual != G.fim: #Passa por todas as operações e atualiza as arestas
            no_atual.feromonio_dir *= evaporacao
            no_atual.feromonio_nao_dir *= evaporacao
            no_atual = no_atual.direcionadas[0] #Próxima operação do job

def soma_contribuicao(formigas): #Soma quanto deve ser adicionado para cada aresta
    contribuicao_por_aresta = {}
    for f in formigas: #Para toda formiga
        no_atual = G.inicio
        for prox_no in f.tabu: #Para todas as arestas da solução
            if (no_atual, prox_no) not in contribuicao_por_aresta: #Se aresta (nó de saída, nó de chegada) não está no dicionário
                contribuicao_por_aresta[(no_atual, prox_no)] = 0 #Coloca com contribuição 0
            contribuicao_por_aresta[(no_atual, prox_no)] += f.contribuicao #Soma a contribuição da formiga na aresta
            no_atual = prox_no #Próxima aresta
    return contribuicao_por_aresta #Retorna dicionário

def aumenta_feromonio(formigas): #Aumenta o feromônio das arestas
    evaporacao_todas_arestas()
    
    contribuicao_por_aresta = soma_contribuicao(formigas) #Pega contribuição por aresta
    for aresta in contribuicao_por_aresta: #Para cada aresta
        saida = aresta[0] #Nó de saída
        chegada = aresta[1] #Nó de chegada
        if chegada in saida.direcionadas: #Se está nas direcionadas
            index = np.where(saida.direcionadas == chegada)[0] #Adiciona feromônio no índice certo (pode não ser 0 no nó inicial)
            saida.feromonio_dir[index] = contribuicao_por_aresta[aresta] + saida.feromonio_dir[index]
        else: #Se estiver nas não direcionadas
            index = np.where(saida.nao_direcionadas == chegada)[0] #Adiciona feromônio no índice certo
            saida.feromonio_nao_dir[index] = contribuicao_por_aresta[aresta] + saida.feromonio_nao_dir[index]

## Ciclos

In [12]:
def cria_pop(): #Cria população de formigas
    solucoes = np.array([])
    while len(solucoes) < num_ants: #Cria todas as formigas e colocar no array
        formiga = Ant()
        formiga.constroi()
        solucoes = np.append(solucoes, formiga)
    return solucoes

def ciclo(): #Executa cada ciclo
    formigas = cria_pop() #População de formigas
    
    min_makespan = -1
    for f in formigas: #Acha a formiga com menos makespan
        if f.makespan == otimo:
            encontrou_otimo += 1
            
        if (min_makespan > f.makespan) or (min_makespan == -1):
            min_makespan = f.makespan
            min_formiga = f
    
    aumenta_feromonio(formigas) #Aumenta feromônios nas arestas
    
    return min_formiga #Retorna a melhor formiga

## Algoritmo

In [13]:
media = 0
d_padrao = 0
def algoritmo(): #Processa o algoritmo
    global media, d_padrao
    min_makespan = -1
    makespans = np.array([])
    for c in range(num_ciclos):
        melhor_ciclo = ciclo() #Melhor formiga de cada ciclo
        makespans = np.append(makespans, melhor_ciclo.makespan) #Salva os makespans por ciclo
        
        if min_makespan > melhor_ciclo.makespan or min_makespan == -1: #Se makespan for menor que a menor atual
            min_formiga = melhor_ciclo
            min_makespan = melhor_ciclo.makespan
    
    media = np.average(makespans)
    d_padrao = np.std(makespans)
    return min_formiga #Retorna a melhor formiga de todos os ciclos

melhor_plano = algoritmo()

In [14]:
melhor_plano

[(2, 14, 73) (2, 3, 82) (2, 1, 23) (2, 12, 62) (2, 6, 88) (2, 5, 21)
 (2, 8, 65) (2, 11, 70) (2, 7, 53) (2, 10, 81) (13, 10, 98) (13, 12, 34)
 (4, 12, 97) (4, 5, 7) (4, 9, 96) (0, 9, 65) (6, 9, 15) (6, 7, 82)
 (6, 6, 21) (6, 14, 53) (6, 3, 72) (6, 13, 49) (13, 13, 52) (13, 4, 26)
 (13, 1, 28) (14, 1, 70) (14, 10, 17) (14, 6, 90) (14, 12, 67) (14, 4, 14)
 (14, 8, 23) (14, 3, 21) (13, 3, 39) (7, 3, 54) (7, 7, 24) (7, 4, 14)
 (7, 8, 38) (7, 5, 36) (7, 2, 52) (10, 2, 54) (10, 0, 22) (9, 0, 87)
 (12, 0, 49) (12, 13, 83) (12, 3, 73) (12, 6, 82) (12, 1, 82) (3, 1, 12)
 (3, 6, 51) (3, 7, 33) (3, 4, 15) (3, 14, 72) (3, 10, 98) (11, 10, 92)
 (11, 14, 36) (11, 4, 22) (4, 4, 15) (4, 14, 73) (7, 14, 55) (12, 14, 92)
 (12, 11, 73) (12, 4, 31) (12, 10, 35) (12, 9, 54) (12, 5, 7) (12, 8, 37)
 (12, 7, 72) (12, 2, 52) (12, 12, 76) (7, 12, 37) (7, 11, 48) (7, 0, 93)
 (7, 13, 60) (7, 10, 70) (7, 1, 23) (7, 6, 23) (7, 9, 83) (3, 9, 94)
 (3, 5, 12) (9, 5, 12) (9, 7, 80) (9, 4, 50) (9, 10, 48) (9, 12, 90)
 (

In [15]:
print('Distância do ótimo:', melhor_plano.makespan - otimo)
print('Encontrou o ótimo:', encontrou_otimo, 'vezes')
print('Média por ciclo:', media)
print('Desvio padrão por ciclo:', d_padrao)

Distância do ótimo: 797
Encontrou o ótimo: 0 vezes
Média por ciclo: 2066.1857142857143
Desvio padrão por ciclo: 13.793676457133598
